In [6]:
import numpy as np
import torch
from utils import pre_processing

# integrated gradients
def integrated_gradients(inputs, model, target_label_idx, predict_and_gradients, baseline, steps=50):
    if baseline is None:
        baseline = 0 * inputs 
    # scale inputs and compute gradients
    scaled_inputs = [baseline + (float(i) / steps) * (inputs - baseline) for i in range(0, steps + 1)]
    grads, _ = predict_and_gradients(scaled_inputs, model, target_label_idx)
    avg_grads = np.average(grads[:-1], axis=0)
    avg_grads = np.transpose(avg_grads, (1, 2, 0))
    delta_X = (pre_processing(inputs) - pre_processing(baseline)).detach().squeeze(0).cpu().numpy()
    delta_X = np.transpose(delta_X, (1, 2, 0))
    integrated_grad = delta_X * avg_grads
    return integrated_grad

def random_baseline_integrated_gradients(inputs, model, target_label_idx, predict_and_gradients, steps, num_random_trials):
    all_intgrads = []
    for i in range(num_random_trials):
        integrated_grad = integrated_gradients(inputs, model, target_label_idx, predict_and_gradients, \
                                                baseline=255.0 *np.random.random(inputs.shape), steps=steps)
        all_intgrads.append(integrated_grad)
        print('the trial number is: {}'.format(i))
    avg_intgrads = np.average(np.array(all_intgrads), axis=0)
    return avg_intgrads


In [7]:
import torch
import torch.nn.functional as F
import cv2
import numpy as np

def calculate_outputs_and_gradients(inputs, model, target_label_idx):
    # do the pre-processing
    predict_idx = None
    gradients = []
    for input in inputs:
        input = pre_processing(input)
        output = model(input)
        output = F.softmax(output, dim=1)
        if target_label_idx is None:
            target_label_idx = torch.argmax(output, 1).item()
        index = np.ones((output.size()[0], 1)) * target_label_idx
        index = torch.tensor(index, dtype=torch.int64)
        output = output.gather(1, index)
        # clear grad
        model.zero_grad()
        output.backward()
        gradient = input.grad.detach().cpu().numpy()[0]
        gradients.append(gradient)
    gradients = np.array(gradients)
    return gradients, target_label_idx

def pre_processing(obs):
    mean = np.array([0.485, 0.456, 0.406]).reshape([1, 1, 3])
    std = np.array([0.229, 0.224, 0.225]).reshape([1, 1, 3])
    obs = obs / 255
    obs = (obs - mean) / std
    obs = np.transpose(obs, (2, 0, 1))
    obs = np.expand_dims(obs, 0)
    obs = np.array(obs)
    torch_device = torch.device('cpu')
    obs_tensor = torch.tensor(obs, dtype=torch.float32, device=torch_device, requires_grad=True)
    return obs_tensor

# generate the entire images
def generate_entrie_images(img, img_grad, img_grad_overlay, img_integrad, img_integrad_overlay):
# Print dimensions of images
    print(f"Original Image Shape: {img.shape}")
    print(f"Gradient Image Shape: {img_grad.shape}")
    print(f"Gradient Overlay Image Shape: {img_grad_overlay.shape}")
    print(f"Integrated Gradient Image Shape: {img_integrad.shape}")
    print(f"Integrated Gradient Overlay Image Shape: {img_integrad_overlay.shape}")
    blank = np.ones((img_grad.shape[0], 10, 3), dtype=np.uint8) * 255
    blank_hor = np.ones((10, 20 + img_grad.shape[0] * 3, 3), dtype=np.uint8) * 255
    upper = np.concatenate([img[:, :, (2, 1, 0)], blank, img_grad_overlay, blank, img_grad], 1)
    down = np.concatenate([img[:, :, (2, 1, 0)], blank, img_integrad_overlay, blank, img_integrad], 1)
    total = np.concatenate([upper, blank_hor, down], 0)
    total = cv2.resize(total, (550, 364))

    return total


In [8]:
import numpy as np
import cv2

G = [0, 255, 0]
R = [255, 0, 0]

def convert_to_gray_scale(attributions):
    return np.average(attributions, axis=2)

def linear_transform(attributions, clip_above_percentile=99.9, clip_below_percentile=70.0, low=0.2, plot_distribution=False):
    m = compute_threshold_by_top_percentage(attributions, percentage=100-clip_above_percentile, plot_distribution=plot_distribution)
    e = compute_threshold_by_top_percentage(attributions, percentage=100-clip_below_percentile, plot_distribution=plot_distribution)
    transformed = (1 - low) * (np.abs(attributions) - e) / (m - e) + low
    transformed *= np.sign(attributions)
    transformed *= (transformed >= low)
    transformed = np.clip(transformed, 0.0, 1.0)
    return transformed

def compute_threshold_by_top_percentage(attributions, percentage=60, plot_distribution=True):
    if percentage < 0 or percentage > 100:
        raise ValueError('percentage must be in [0, 100]')
    if percentage == 100:
        return np.min(attributions)
    flat_attributions = attributions.flatten()
    attribution_sum = np.sum(flat_attributions)
    sorted_attributions = np.sort(np.abs(flat_attributions))[::-1]
    cum_sum = 100.0 * np.cumsum(sorted_attributions) / attribution_sum
    threshold_idx = np.where(cum_sum >= percentage)[0][0]
    threshold = sorted_attributions[threshold_idx]
    if plot_distribution:
        raise NotImplementedError 
    return threshold

def polarity_function(attributions, polarity):
    if polarity == 'positive':
        return np.clip(attributions, 0, 1)
    elif polarity == 'negative':
        return np.clip(attributions, -1, 0)
    else:
        raise NotImplementedError

def overlay_function(attributions, image):
    return np.clip(0.7 * image + 0.5 * attributions, 0, 255)

def visualize(attributions, image, positive_channel=G, negative_channel=R, polarity='positive', \
                clip_above_percentile=99.9, clip_below_percentile=0, morphological_cleanup=False, \
                structure=np.ones((3, 3)), outlines=False, outlines_component_percentage=90, overlay=True, \
                mask_mode=False, plot_distribution=False):
    if polarity == 'both':
        raise NotImplementedError

    elif polarity == 'positive':
        attributions = polarity_function(attributions, polarity=polarity)
        channel = positive_channel
    
    # convert the attributions to the gray scale
    attributions = convert_to_gray_scale(attributions)
    attributions = linear_transform(attributions, clip_above_percentile, clip_below_percentile, 0.0, plot_distribution=plot_distribution)
    attributions_mask = attributions.copy()
    if morphological_cleanup:
        raise NotImplementedError
    if outlines:
        raise NotImplementedError
    attributions = np.expand_dims(attributions, 2) * channel
    if overlay:
        if mask_mode == False:
            attributions = overlay_function(attributions, image)
        else:
            attributions = np.expand_dims(attributions_mask, 2)
            attributions = np.clip(attributions * image, 0, 255)
            attributions = attributions[:, :, (2, 1, 0)]
    return attributions


In [ ]:
import os
import glob
import cv2
import numpy as np
from torchvision import models
from torchvision.models import VGG19_Weights
# utils, integrated_gradients ve visualization modüllerinin doğru import edildiğinden emin olun.
from utils import calculate_outputs_and_gradients, generate_entrie_images
from integrated_gradients import random_baseline_integrated_gradients
from visualization import visualize

# VGG19 modelini oluştur ve değerlendirme moduna al
model = models.vgg19(weights=VGG19_Weights.DEFAULT)
model.eval()

# İşlenecek görüntülerin bulunduğu klasör
image_directory = 'example/'  # Klasör adı doğru mu kontrol et
# Görüntü dosya uzantısı
image_extension = '*.jpg'

# Sonuçları saklamak için dizinleri kontrol et ve oluştur
results_directory = 'results/vgg19'
if not os.path.exists(results_directory):
    os.makedirs(results_directory)

# Belirtilen klasördeki tüm görüntü dosyalarını listele
image_files = glob.glob(os.path.join(image_directory, image_extension))

# Her bir görüntü dosyası için döngü
for img_file in image_files:
    # Görüntüyü oku ve ön işle
    img = cv2.imread(img_file)
    img = cv2.resize(img, (224, 224))  # VGG19 için uygun boyuta getir
    img = img.astype(np.float32)
    img = img[:, :, (2, 1, 0)]  # BGR'dan RGB'ye çevir
    
    # Gradyanları ve etiket indeksini hesapla
    gradients, label_index = calculate_outputs_and_gradients([img], model, None)
    # Kategori ismini al
    category_name = VGG19_Weights.DEFAULT.meta["categories"][label_index]
    
    # Entegre gradyanları hesapla
    attributions = random_baseline_integrated_gradients(img, model, label_index, calculate_outputs_and_gradients,
                                                        steps=50, num_random_trials=6)
    # Görselleştirme
    img_integrated_gradient = visualize(attributions, img, clip_above_percentile=99, clip_below_percentile=0,
                                                overlay=False)

    # Sonuçları kaydetmek için klasör oluştur
    save_directory = os.path.join(results_directory, category_name)
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
    
    # İşlenmiş görüntüyü kaydet
    img_name = os.path.basename(img_file).split('.')[0]  # Dosya adından uzantıyı çıkar
    save_path = os.path.join(save_directory, f'{img_name}_integrated_gradient.jpg')
    cv2.imwrite(save_path, np.uint8(img_integrated_gradient))

the trial number is: 0
the trial number is: 1
the trial number is: 2
the trial number is: 3
the trial number is: 4
the trial number is: 5
the trial number is: 0
the trial number is: 1
the trial number is: 2
